In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
%matplotlib inline

## Camera Calibration with Open CV

In [ ]:
#Arrays to store objects points and image points from all the images

objpoints = [] #3D points in real world space
imgpoints = [] #2D points in image plane

#Prepare object points, like (0,0,0) ,(1,0,0),(2,0,0) .....,(8,5,0)
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) #x,y coordinates

# Make a list of calibration images

images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    
    img = cv2.imread(fname)
    img2 = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    # If found, add object points, image points
    if ret == True:    
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img2 = cv2.drawChessboardCorners(img2, (9,6), corners, ret)

        
    f, (ax1,ax2) = plt.subplots(1, 2, figsize=(24,8))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('original_Image - ' + fname[11:], fontsize=30)
    ax2.imshow(img2)
    ax2.set_title('corners_detected', fontsize=30)
       
    #You may also uncomment the following line for generate output files
    #plt.savefig('camera_cal/ChessboardCorners/'+'ChessboardCorners_'+fname[11:])

    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
chess_img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, chess_img_size,None,None)

In [ ]:
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    dst_image = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)

    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,5.5))
    ax1.imshow(img)
    ax1.set_title('Original Image- ' + fname[11:], fontsize=30)
    ax2.imshow(dst_image)
    ax2.set_title('Undistorted Image', fontsize=30)
   
    #You may also uncomment the following line for generate output files
    #plt.savefig('camera_cal/Chessboard_Undistortion/'+'Cb_Undistortion_'+fname[11:])

    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Applying function to Undistorted the images.

In [ ]:
def Undistorted_Images(img,mtx,dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist = Undistorted_Images(img,mtx,dist)
   
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,8))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original image - ' + fname[12:], fontsize=30)
    ax2.imshow(undist)
    ax2.set_title('undistorted version', fontsize=30)
    
    #You may also uncomment the following line for generate output files
    plt.savefig('output_images/Undistorted_images/'+'Undist_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
plt.imshow(undist)

## Testing Gradient Threshold

In [ ]:
def abs_sobel_thresh(img, orient='x',thresh = (50,150), kernel_size = 7 ):
    
    # Apply the following steps to img
    #1) Convert to grayscale
     
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    #Applies a Gaussian Noise kernel
    blur =  cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(blur, cv2.CV_64F, 1, 0)
    if orient == 'y':
        sobel = cv2.Sobel(blur, cv2.CV_64F, 0, 1)

        
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # 6) Return this mask as your binary_output image
    grad_binary = sbinary

    return grad_binary

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)
     
    grad_binary_x = abs_sobel_thresh(undist, orient='x',thresh = (30,120), kernel_size = 5 )
    grad_binary_y = abs_sobel_thresh(undist, orient='y',thresh = (30,120), kernel_size = 5 )

    
    f, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(24,6))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - ' + fname[12:], fontsize=30)
    ax2.imshow(grad_binary_x,cmap = 'gray')
    ax2.set_title('abs_sobel_X_binary', fontsize=30)
    ax3.imshow(grad_binary_y,cmap = 'gray')
    ax3.set_title('abs_sobel_Y_binary', fontsize=30)
    
    #You may also uncomment the following line for generate output files
    #plt.savefig('output_images/Abs_Sobel/'+'Abs_Sobel_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def mag_thresh(img,sobel_kernel=15, mag_thresh=(0, 255), blur_kernel =15):
    
    # Apply the following steps to img
    # 1) Convert to grayscale and Applies a Gaussian Noise kernel
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur =  cv2.GaussianBlur(gray, (blur_kernel, blur_kernel), 0)
    
    # 2) Take the gradient in x and y separately
    
    sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1,ksize=sobel_kernel)

    # 3) Calculate the magnitude 
    
    gradmag = np.sqrt(sobelx**2+sobely**2)

    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8

    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    # 5) Create a binary mask where mag thresholds are met
    
    binary_mask = np.zeros_like(gradmag)
    binary_mask[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1    
    
    # 6) Return this mask as your binary_output image
    
    mag_binary = binary_mask 
    
    return mag_binary

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)
    mag_binary = mag_thresh(undist,sobel_kernel=15, mag_thresh=(30, 120), blur_kernel =5)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+ fname[12:], fontsize=30)
    ax2.imshow(mag_binary,cmap = 'gray')
    ax2.set_title('mag_thresh_binary result', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/magthresh/'+'magthresh_binary_'+fname[12:])
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    


In [ ]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    
    # 3) Take the absolute value of the x and y gradients

    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    
    absgraddir = np.arctan2(abs_sobely, abs_sobelx)
    
    # 5) Create a binary mask where direction thresholds are met
    
    sbinary = np.zeros_like(absgraddir)
    sbinary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line

    dir_binary = sbinary
    
    return dir_binary

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)
    dir_binary = dir_threshold(undist, sobel_kernel=11, thresh=(0.6, 1.2))
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+fname[12:], fontsize=30)
    ax2.imshow(dir_binary,cmap = 'gray')
    ax2.set_title('dir_threshold_binary', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/dir_threshold/'+'dir_threshold_'+fname[12:])
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Testing Color Threshold

In [ ]:
def rgb_binary(img,channel,thresh =(200,255)):

    
    if channel == 'r':
        channel = img[:,:,0]
    if channel == 'g':
        channel = img[:,:,1]  
    if channel == 'b':
        channel = img[:,:,2]  
    # 2) Apply a threshold to the S channel
    
    binary = np.zeros_like(channel)
    binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1

    
    # 3) Return a binary image of threshold result
    
    #binary_output = np.copy(img) # placeholder line
    rgb_binary =  binary
    
    return rgb_binary

In [ ]:
images = glob.glob('test_images/*.jpg')

rgb_thresh =(200,255)

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)
    rgb_binary_r = rgb_binary(undist,'r',rgb_thresh)
    rgb_binary_g = rgb_binary(undist,'g',rgb_thresh)
    rgb_binary_b = rgb_binary(undist,'b',rgb_thresh)
    
    
    f, (ax1, ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(24, 4))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+fname[12:], fontsize=20)
    ax2.imshow(rgb_binary_r,cmap = 'gray')
    ax2.set_title('R channel_binary', fontsize=20)
    ax3.imshow(rgb_binary_g,cmap = 'gray')
    ax3.set_title('G channel_binary', fontsize=20)
    ax4.imshow(rgb_binary_b,cmap = 'gray')
    ax4.set_title('B channel_binary', fontsize=20)

    #You may also uncomment the following line for generate output files
    #plt.savefig('output_images/rgb_binary/'+'rgb_binary_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def hls_binary(img,channel,thresh =( 0,255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if channel == 'h':
        channel = hls[:,:,0]
    if channel == 'l':
        channel = hls[:,:,1]  
    if channel == 's':
        channel = hls[:,:,2]  
    # 2) Apply a threshold to the S channel
    
    binary = np.zeros_like(channel)
    binary[(channel >= thresh[0]) & (channel <= thresh[1])] = 1
    
    # 3) Return a binary image of threshold result
    
    #binary_output = np.copy(img) # placeholder line
    hls_binary =  binary
    
    return hls_binary

In [ ]:
images = glob.glob('test_images/*.jpg')

hls_thresh =(180,255)

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)
    hls_binary_h = hls_binary(undist,'h',hls_thresh)
    hls_binary_l = hls_binary(undist,'l',hls_thresh)
    hls_binary_s = hls_binary(undist,'s',hls_thresh)
    
    
    f, (ax1, ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(24, 4))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+fname[12:], fontsize=20)
    ax2.imshow(hls_binary_h,cmap = 'gray')
    ax2.set_title('h channel_binary', fontsize=20)
    ax3.imshow(hls_binary_l,cmap = 'gray')
    ax3.set_title('l channel_binary', fontsize=20)
    ax4.imshow(hls_binary_s,cmap = 'gray')
    ax4.set_title('s channel_binary', fontsize=20)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/hls_binary/'+'hls_binary_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Combining gradient and color thresholds

In [ ]:
def color_and_gradient(img,sx_thresh,rgb_thresh,hls_thresh):
    
    gradx = abs_sobel_thresh(img,orient='x', thresh = (sx_thresh[0], sx_thresh[1]),kernel_size = 5)
    r_binary = rgb_binary(img,'r', thresh = (rgb_thresh[0],rgb_thresh[1]))
    s_binary = hls_binary(img,'s', thresh = (hls_thresh[0],hls_thresh[1]))
    
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(gradx ),gradx ,s_binary  )) * 255
    
    combined_binary = np.zeros_like(gradx)
    combined_binary[(r_binary == 1) |(s_binary == 1) & (gradx == 1)] = 1
  
    return gradx,r_binary,s_binary,combined_binary 

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)

    gradx,r_binary,s_binary,combined_binary = color_and_gradient(undist,sx_thresh = (30,120),rgb_thresh = (220,255),hls_thresh = (180,255))
    
    f, (ax1, ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(24, 4))
    f.tight_layout()

    ax1.imshow(combined_binary,cmap = 'gray')
    ax1.set_title('combined - '+fname[12:], fontsize=20)        
    ax2.imshow(r_binary,cmap = 'gray')
    ax2.set_title('R channel', fontsize=20)  
    ax3.imshow(gradx,cmap = 'gray')
    ax3.set_title('abs_sobel_X_binary', fontsize=20)
    ax4.imshow(s_binary,cmap = 'gray')
    ax4.set_title('s channel_binary', fontsize=20)

    #You may also uncomment the following line for generate output files    
    plt.savefig('output_images/combined_binary/'+'combined_binary_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Creating functions to Mask the lanes lines image

In [ ]:
def apply_mask (img):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    imshape = img.shape
    vertices = np.array([[(100,imshape[0]),(550,450), (imshape[1]-520, 450), (imshape[1]-60,imshape[0])]], dtype=np.int32)        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



In [ ]:
images = glob.glob('test_images/straight_lines*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)

    marked_image = apply_mask (undist)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+fname[12:], fontsize=30)
    ax2.imshow(marked_image,cmap = 'gray')
    ax2.set_title('Masked_image', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/masked/'+'masked_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Applying extended lines on images

In [ ]:
def draw_Extended_lines(img, lines, color=[255, 0, 0], thickness=15):

    #The try statement was created due video test results. In some cases the video was not generated,
    #because some point was not identified and the algorithm could not calculate the average due to occur
    #the division by zero. ( avg_x1 , .... , avg_y4).
    
    try:
        
        # generating lists with the coordinate value for each point.
        imshape = img.shape
        y_lower = imshape[0]
        x1l = []
        y1l = []
        x2l = []
        y2l = [] 
        x3l = []
        y3l = []
        x4l = []
        y4l = [] 
        for line in lines:       
            for x1,y1,x2,y2 in line:
                if((y2-y1)/(x2-x1))<0:
                    x1l.append(x1)
                    y1l.append(y1)
                    x2l.append(x2)
                    y2l.append(y2)
                else:
                    x3l.append(x1)
                    y3l.append(y1)
                    x4l.append(x2)
                    y4l.append(y2)                

        # Calculate the average for eache coordinate - 8 coordinates / 4 points / 2 lines.
                    
        avg_x1 = int(sum(x1l)/len(x1l))
        avg_y1 = int(sum(y1l)/len(y1l))
        avg_x2 = int(sum(x2l)/len(x2l))                
        avg_y2 = int(sum(y2l)/len(y2l))                
        avg_x3 = int(sum(x3l)/len(x3l))
        avg_y3 = int(sum(y3l)/len(y3l))
        avg_x4 = int(sum(x4l)/len(x4l))                
        avg_y4 = int(sum(y4l)/len(y4l))


        #Applying Analytic geometry concept to accomplish the goals
        #The "General Form" of the equation of a straight line Ax + By + C = 0
        
        a1 = avg_y1-avg_y2
        b1 = avg_x2-avg_x1
        c1 = (avg_x1*avg_y2)-(avg_y1*avg_x2)

        a2 = avg_y3-avg_y4
        b2 = avg_x4-avg_x3
        c2 = (avg_x3*avg_y4)-(avg_y3*avg_x4)   


        #For the upper point will be considered Y coordinate at the limite of the masked imag = 450 +20 :
        
        y_upper = 500
        
        x_upper_line_left = int((-b1*y_upper-c1)/a1)
        x_upper_line_right = int((-b2*y_upper-c2)/a2)
        

        #calculating the Xpoint coordinate for lower point from the lines.
        # The Y coordinate is equal to the image size. "y_lower = imshape[1]"

        x_lower_line_left = int((-b1*y_lower-c1)/a1)
        x_lower_line_right = int((-b2*y_lower-c2)/a2)

        p1 = [x_lower_line_left,y_lower]
        p2 = [x_upper_line_left,y_upper]        
        p3 = [x_upper_line_right,y_upper]        
        p4 = [x_lower_line_right,y_lower]        
        
        # generating extended lines (left and Right)
        cv2.line(img,(p1[0],p1[1]),(p2[0],p2[1]), color, thickness)
        cv2.line(img,(p3[0],p3[1]),(p4[0],p4[1]), color, thickness)
                       
    except:
        pass 
    
    
def hough_Extended_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_Extended_lines(line_img, lines)
    return line_img 

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):

    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
def draw_extended_lines(img):
    undist = Undistorted_Images(img,mtx,dist)
    gradx,r_binary,s_binary,combined_binary = color_and_gradient(undist,sx_thresh = (30,120),rgb_thresh = (220,255),hls_thresh = (180,255))

    masked_image = apply_mask (combined_binary)

    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 20 #minimum number of pixels making up a line
    max_line_gap = 300  # maximum gap in pixels between connectable line segments

    extended_lines_edges = hough_Extended_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    extended_weighted_img = weighted_img(extended_lines_edges, undist, α=0.3, β=1, γ=1)
    
    return extended_weighted_img

In [ ]:
images = glob.glob('test_images/straight_lines*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)

    undist = Undistorted_Images(img,mtx,dist)

    lines = draw_extended_lines (undist)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+fname[12:], fontsize=30)
    ax2.imshow(lines,cmap = 'gray')
    ax2.set_title('Lines added', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/get_coordinate_lines/'+'get_coordinate_lines_'+fname[12:])    
    
#get_coordinate_lines    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Getting coordinates to support warp function.

In order to construct a reliable matrix to warp the images, it will be considered the images straight_lines 1 and 2 , because the lines generated programmatically, show good results. As second step, it will be calculated the tangent ($\Delta$y/$\Delta$x) for the left line and right line

To ensure that will be considered the whole line lenght , from the 6 images available will be considered the "x" coordinates minimun and  maximum in the lower region of the pictures  and it will be applied the tangent angle for both lines inverting the direction on each case.

In [ ]:
# It will be created a function simlar from the function that have draw the lines,
# but the functin will return the coordinates for the 4 points .


def get_coordinates(line_img,lines):

    #The try statement was created due video test results. In some cases the video was not generated,
    #because some point was not identified and the algorithm could not calculate the average due to occur
    #the division by zero. ( avg_x1 , .... , avg_y4).
    
    try:
        
        # generating lists with the coordinate value for each point.
        imshape = line_img.shape
        y_lower = imshape[0]
        x1l = []
        y1l = []
        x2l = []
        y2l = [] 
        x3l = []
        y3l = []
        x4l = []
        y4l = [] 
        for line in lines:       
            for x1,y1,x2,y2 in line:
                if((y2-y1)/(x2-x1))<0:
                    x1l.append(x1)
                    y1l.append(y1)
                    x2l.append(x2)
                    y2l.append(y2)
                else:
                    x3l.append(x1)
                    y3l.append(y1)
                    x4l.append(x2)
                    y4l.append(y2)                

        # Calculate the average for eache coordinate - 8 coordinates / 4 points / 2 lines.
                    
        avg_x1 = int(sum(x1l)/len(x1l))
        avg_y1 = int(sum(y1l)/len(y1l))
        avg_x2 = int(sum(x2l)/len(x2l))                
        avg_y2 = int(sum(y2l)/len(y2l))                
        avg_x3 = int(sum(x3l)/len(x3l))
        avg_y3 = int(sum(y3l)/len(y3l))
        avg_x4 = int(sum(x4l)/len(x4l))                
        avg_y4 = int(sum(y4l)/len(y4l))


        #Applying Analytic geometry concept to accomplish the goals
        #The "General Form" of the equation of a straight line Ax + By + C = 0
        
        a1 = avg_y1-avg_y2
        b1 = avg_x2-avg_x1
        c1 = (avg_x1*avg_y2)-(avg_y1*avg_x2)

        a2 = avg_y3-avg_y4
        b2 = avg_x4-avg_x3
        c2 = (avg_x3*avg_y4)-(avg_y3*avg_x4)   


        #For the upper point will be considered Y coordinate at the limite of the masked imag = 450 +20 :
        
        y_upper = 450
        
        x_upper_line_left = int((-b1*y_upper-c1)/a1)
        x_upper_line_right = int((-b2*y_upper-c2)/a2)
        

        #calculating the Xpoint coordinate for lower point from the lines.
        # The Y coordinate is equal to the image size. "y_lower = imshape[1]"

        x_lower_line_left = int((-b1*y_lower-c1)/a1)
        x_lower_line_right = int((-b2*y_lower-c2)/a2)

        p1 = [x_lower_line_left,y_lower]
        p2 = [x_upper_line_left,y_upper]        
        p3 = [imshape[1]-x_upper_line_left,y_upper]        
        p4 = [imshape[1]-x_lower_line_left,y_lower]        
        
        return p1,p2,p3,p4
    
    except:
        pass 
    
def hough_Extended_lines_2 (img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    p1,p2,p3,p4 = get_coordinates(line_img, lines)
    
    return p1,p2,p3,p4

In [ ]:
# Getting the 4 points coordinates for all pictures available.

def getting_4_points_coordinates (img):
    undist = Undistorted_Images(img,mtx,dist)
    gradx,r_binary,s_binary,combined_binary = color_and_gradient(undist,sx_thresh = (50,150),rgb_thresh = (220,255),hls_thresh = (180,255))

    masked_image = apply_mask (combined_binary)

    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 20 #minimum number of pixels making up a line
    max_line_gap = 300  # maximum gap in pixels between connectable line segments

    p1,p2,p3,p4 = hough_Extended_lines_2(masked_image, rho, theta, threshold, min_line_len, max_line_gap)

    m_points = [p1,p2,p3,p4]
    
    return m_points

In [ ]:
#Calculating the average for the tangente for the left and right lines for the files with straight_lines.

straight_lines_files = ['straight_lines1.jpg','straight_lines2.jpg']

tan_value =[]

for i in straight_lines_files:

    img = mpimg.imread ('test_images/'+i)
    m_points = getting_4_points_coordinates (img)

    tan_line_left = (m_points[0][1]-m_points[1][1])/(m_points[1][0]-m_points[0][0])
    tan_value.append(tan_line_left)
    tan_line_right = (m_points[3][1]-m_points[2][1])/(m_points[3][0]-m_points[2][0])
    tan_value.append(tan_line_right)

avg_tan = sum(tan_value)/len(tan_value)

avg_tan

In [ ]:
#In order to ensure the lane lines, It Will be checked the minimum and maximum X coordinate available
#in a set of images.


images = glob.glob('test_images/*.jpg')

min_max_X_lower_coordinates = [1000,0]

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    m_points = getting_4_points_coordinates (img)
    
    if m_points[0][0] < min_max_X_lower_coordinates[0]:
        min_max_X_lower_coordinates[0] = m_points[0][0]
    if m_points[3][0]> min_max_X_lower_coordinates[1]:
        min_max_X_lower_coordinates[1] = m_points[3][0]
        
min_max_X_lower_coordinates 

In [ ]:
#checking if the reference line point must be from left , right line or any line.
    
img = mpimg.imread ('test_images/test1.jpg')
imshape = img.shape
    
    
if (imshape[1]- min_max_X_lower_coordinates[1] ) == min_max_X_lower_coordinates [0]:
    print ("any line could be used as reference")    

elif (imshape[1] - min_max_X_lower_coordinates[1] ) > min_max_X_lower_coordinates [0]:
    print ("use right lower point (P4x) as reference")
    
else:
    print ("use left lower point (P1x) as reference")


In [ ]:
# apply the avg_tan previously calculated for the min and max lower "X" coordinates (p1,p2),
# and fixing the "Y" coordinate =450 , it will be calculated the X cordinate for the upper points (p2 , p3).

#The new matrix with this 4 new points it will considered as the source information for the warped function.

img = mpimg.imread ('test_images/straight_lines1.jpg')

imshape = img.shape

y_upper = 460
y_lower = imshape[0]


# In order to keep a symmetry, the P1 "X" position will be adjusted considering the P4 "X"value.

p1_x = (min_max_X_lower_coordinates [0])

p4_x = (min_max_X_lower_coordinates [1])

p2_x = (((y_lower - y_upper) / avg_tan)+ p1_x)

p3_x = (p4_x - ((y_lower - y_upper) / avg_tan))

np1,np2,np3,np4 = [p1_x,y_lower],[p2_x,y_upper ],[p3_x,y_upper ],[p4_x,y_lower]

np1,np2,np3,np4

In [ ]:
# Defining the source matrix (src) and destination

#the coefficient applied was created to tuning the warped image

coef = 55
    
np2[0] = np2[0]+coef
np3[0] = np3[0]-coef

src = np.float32([np1,np2,np3,np4])

offset= 220
height, width = undist.shape[0], undist.shape[1]
dst = np.float32([[offset,height],[offset,0],[width-offset,0],[width-offset,height]])


## Defining function to warp images

In [ ]:
## Defining Matrixs M and Minv.

M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst,src)


In [ ]:
def warp_images (img):
    
    undist = Undistorted_Images(img,mtx,dist)
    warped = cv2.warpPerspective(undist, M ,(imshape[1], imshape[0]))

    return undist,warped

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist,warped = warp_images (img)
    gradx,r_binary,s_binary,combined_binary = color_and_gradient(warped,sx_thresh = (30,120),rgb_thresh = (220,255),hls_thresh = (180,255))
    
    f, (ax1, ax2,ax3) = plt.subplots(1,3, figsize=(24, 5.5))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted - '+str(fname[12:]), fontsize=25)
    ax2.imshow(warped,cmap = 'gray')
    ax2.set_title('warped ', fontsize=25)
    ax3.imshow(combined_binary,cmap = 'gray')
    ax3.set_title('warped Binary  ', fontsize=25)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/warp_images/'+'warped_'+fname[12:])    
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Defining Pipeline to create warped binary image and plot a histogram.

In [ ]:
def Pipeline_warped_binary (img):

    undist,warped = warp_images (img)
    gradx,r_binary,s_binary,combined_binary = color_and_gradient(warped,sx_thresh = (30,120),rgb_thresh = (220,255),hls_thresh = (180,255))
    return undist,combined_binary

def hist(img):
    
    #imshape = img.shape
    
    img =  (img - np.min(img))/np.ptp(img)
    
    bottom_half =  img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist,binary_warped = Pipeline_warped_binary(img)
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    
    f, (ax1, ax2,ax3) = plt.subplots(1,3, figsize=(22, 5))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted -'+ str(fname[12:]), fontsize=20)
    ax2.imshow(binary_warped,cmap = 'gray')
    ax2.set_title('warped_binary', fontsize=20)
    ax3.plot(histogram)

    #You may also uncomment the following line for generate output files
    #plt.savefig('output_images/histogram/'+'histogram_'+fname[12:])     
    
    plt.subplots_adjust(left=0., right=1, top=1.5, bottom=0.)

## Applying lane curves on warped binary images

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 25
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds =  ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))        

        #pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return  left_fit,right_fit,ploty , left_fitx , right_fitx, out_img



In [ ]:
def draw_lines_warped(binary_warped ,ploty , left_fitx , right_fitx,out_img):
    
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    color_warp = out_img

    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    #cv2.fillPoly(color_warp, np.int_([pts_right]), (0,255, 0))

    
    result = color_warp

  
    
    return result     
    

In [ ]:
def draw_lines_warped(binary_warped ,ploty , left_fitx , right_fitx,out_img):

    margin = 100  
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    return result

## Check values left_fitx , right_fitx to validate fit_polynomial function.

In [ ]:
# In order to support the Advanced lane Finding code  to be appled on videos, it will be created the constantes
#sl_left_fit and sl_right_fit to use as reference. The image will be straight_lines1.jpg.

img = mpimg.imread ('test_images/straight_lines1.jpg')
undist,binary_warped = Pipeline_warped_binary (img)
sl_left_fit,sl_right_fit, ploty , left_fitx , right_fitx, out_img = fit_polynomial(binary_warped)


print (sl_left_fit)
print (sl_right_fit)


In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist,binary_warped = Pipeline_warped_binary (img)
    left_fit,right_fit,ploty , left_fitx , right_fitx, out_img = fit_polynomial(binary_warped)
    result = draw_lines_warped(binary_warped ,ploty , left_fitx , right_fitx,out_img)
    
    f, (ax1, ax2,ax3) = plt.subplots(1,3, figsize=(24, 5.5))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Undistorted -'+ str(fname[12:]), fontsize=30)
    ax2.imshow(binary_warped,cmap = 'gray')
    ax2.set_title('binary_warped', fontsize=30)
    ax3.imshow(result)
    ax3.set_title('Polynomial-Warped', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/polynomial/'+'polynomial_'+fname[12:])    
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## adding curves on original image

In [ ]:
Minv = cv2.getPerspectiveTransform(dst, src)


def draw_lines(warped ,ploty , left_fitx , right_fitx):
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result 

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist,binary_warped = Pipeline_warped_binary (img)
    left_fit,right_fit,ploty , left_fitx , right_fitx,out_img = fit_polynomial(binary_warped)
    result = draw_lines(binary_warped ,ploty , left_fitx , right_fitx)
    
    f, (ax1, ax2) = plt.subplots(1,2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original image -'+ str(fname[12:]), fontsize=30)
    ax2.imshow(result)
    ax2.set_title('detected lane', fontsize=30)
    
    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/warp_back/'+'warp_back_'+fname[12:])
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Defining functions to calculate Vehicle turn radius and the offset position on lane line

In [ ]:
#to accurate the conversion from pixels to meters , It will be used the left_fitx[719] and right_fitx [719]
#generated from the image straight_lines1.jpg the diference between this 2 values is the lane width in pixels.

img = mpimg.imread ('test_images/straight_lines1.jpg')
undist,binary_warped = Pipeline_warped_binary (img)
left_fit,right_fit, ploty , left_fitx , right_fitx, out_img = fit_polynomial(binary_warped)


lane_line_width_pixels = right_fitx[719]- left_fitx [719]

In [ ]:
def measure_curvature_real():
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = float(3.7/lane_line_width_pixels)
    ym_per_pix = float(30/(720)) # meters per pixel in y dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    left_fit,right_fit,ploty ,left_fitx , right_fitx,out_img = fit_polynomial(binary_warped)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    
    left_fitx = left_fitx[::-1]  # Reverse to match top-to-bottom in y
    right_fitx = right_fitx[::-1]  # Reverse to match top-to-bottom in y
    
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)    
    
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    if left_curverad > 1500 or right_curverad > 1500:
        vehicle_curverad = 0
        direction = "straight line"        
    
    else:
    
    
        if left_curverad > right_curverad :
            vehicle_curverad = ((left_curverad - right_curverad)/2 )+ right_curverad
            direction = "to right"
        
        elif left_curverad < right_curverad :
            vehicle_curverad = ((right_curverad - left_curverad )/2) + left_curverad 
            direction = "to left"  
              
        else:
            vehicle_curverad = 0
            direction = "straight line"          
        
    return  vehicle_curverad , direction ,left_fitx , right_fitx


In [ ]:
def measure_offset(img,left_fitx , right_fitx,width_in_pixels):
    
    imshape = img.shape

    # Define conversions in x and y from pixels space to meters
    xm_per_pix = float(3.7/lane_line_width_pixels) # meters per pixel in x dimension      
        
    left_fitx_pos = left_fitx[700]
    right_fitx_pos = right_fitx[700]
    lane_line_center = ((right_fitx_pos - left_fitx_pos)/2)+ left_fitx_pos

    if lane_line_center > imshape[1]/2:
        offset = (lane_line_center - imshape[1]/2)* xm_per_pix
        position = "to left"
    elif lane_line_center < imshape[1]/2:
        offset = (imshape[1]/2 - lane_line_center)* xm_per_pix
        position = "to Right" 
  
       
    else:
        offset = 0
        position = "In the middle"         
    
    return abs(offset) , position  


# # Adding information on the pictures

In [ ]:
def add_info(img , vehicle_curverad , direction,offset,position):
    
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    text1 = 'vehicle turn radius: ' + '{:04.1f}'.format(vehicle_curverad) + 'm' + ' to {}'.format(direction)
    cv2.putText(img, text1, (40,50), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    text2 = 'Offset vehicle on the lane: ' + '{:.2f}'.format(abs(offset)) + 'm' + ' to {}'.format(position)
    cv2.putText(img, text2, (40,100), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    return img   

In [ ]:
images = glob.glob('test_images/*.jpg')

for idx, fname in enumerate(images):

    img = mpimg.imread (fname)
    undist,binary_warped = Pipeline_warped_binary (img)
    left_fit,right_fit,ploty , left_fitx , right_fitx,out_img = fit_polynomial(binary_warped)
    
    result = draw_lines(binary_warped ,ploty , left_fitx , right_fitx)

    offset , position = measure_offset(undist,left_fitx , right_fitx,lane_line_width_pixels)
    vehicle_curverad , direction ,left_fitx , right_fitx = measure_curvature_real()
        
    
    img_text = add_info(result , vehicle_curverad , direction,offset,position)
    
    f, (ax1, ax2) = plt.subplots(1,2, figsize=(24, 8))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original image -'+ str(fname[12:]), fontsize=30)
    ax2.imshow(img_text)
    ax2.set_title('Added Lines + Information', fontsize=30)

    #You may also uncomment the following line for generate output files    
    #plt.savefig('output_images/final_images/'+'final_images_'+fname[12:])    
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Exporting Data to support the code.

In [ ]:
dist_pickle = {}

dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
dist_pickle["M"] = M
dist_pickle["Minv"] = Minv
dist_pickle["lane_line_width_pixels"] = lane_line_width_pixels
dist_pickle["imshape"] = imshape

pickle.dump( dist_pickle, open( 'test_images/info_data.p', "wb" ) )